OpenVINO 준비

In [1]:
%pip install -q "openvino>=2023.1.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 22.8 MB/s eta 0:00:00


In [3]:
!pip install openvino-dev

keras .tf 모델을 Intel OpenVINO IR 파일(xml, bin)로 한다.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import openvino
import openvino as ov
from openvino.tools.mo import convert_model

BASE_PATH = './drive/MyDrive/competition/drone_real/data/'
ov_model = convert_model(BASE_PATH+'bin/mobilenetv2_class8.tf')
ov.save_model(ov_model, BASE_PATH+'bin/mobilenetv2_class8.xml')

### 테스트 파일 목록으로 읽어와서 답안 출력하기.



In [12]:
# --------------------------- Step 1. Initialize OpenVINO Runtime Core ------------------------------------------------
core = ov.Core()

In [13]:
# --------------------------- Step 2. Read a model --------------------------------------------------------------------
# (.xml and .bin files) or (.onnx file)
model = core.read_model(BASE_PATH+'bin/mobilenetv2_class8.xml')

if len(model.inputs) != 1: print('Sample supports only single input topologies')
if len(model.outputs) != 1: print('Sample supports only single output topologies')

In [15]:
import os

directory_path = "./drive/MyDrive/competition/drone_real/data/val"
#directory_path = "./drive/MyDrive/Colab Notebooks/data/test"

# Get a list of all files in the directory
folder_list = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]

# Print the list of files
print("List of files in the directory:")
file_list = []
for folder in folder_list:
    flist = [f for f in os.listdir(directory_path+'/'+folder) if os.path.isfile(os.path.join(directory_path+'/'+folder, f))]
    for file in flist:
        file_list.append(directory_path+'/'+folder+"/"+file)
        print(file)

List of files in the directory:
IMG_7602.jpg
IMG_7595.jpg
IMG_7600.jpg
IMG_7594.jpg
IMG_7605.jpg
IMG_7612.jpg
IMG_7634.jpg
IMG_7642.jpg
IMG_7658.jpg
IMG_7660.jpg
IMG_7659.jpg
IMG_7678.jpg
IMG_7681.jpg
IMG_7689.jpg
IMG_7694.jpg
IMG_7687.jpg
IMG_7683.jpg
IMG_7707.jpg
IMG_7698.jpg
IMG_7713.jpg
IMG_7702.jpg
IMG_7708.jpg
IMG_7716.jpg
IMG_7712.jpg
IMG_7725.jpg
IMG_7736.jpg
IMG_7721.jpg
IMG_7737.jpg
IMG_7749.jpg
IMG_7750.jpg
IMG_7755.jpg
IMG_7774.jpg
IMG_7768.jpg
IMG_7769.jpg
IMG_7760.jpg
IMG_7763.jpg
IMG_7758.jpg
IMG_7783.jpg
IMG_7792.jpg
IMG_7778.jpg
IMG_7784.jpg
IMG_7786.jpg
IMG_7794.jpg
IMG_7796.jpg
IMG_2837.jpg
IMG_2838.jpg
IMG_2863.jpg
IMG_2859.jpg
IMG_2846.jpg
IMG_2851.jpg
IMG_2885.jpg
IMG_2881.jpg
IMG_2869.jpg
IMG_3862.jpg
IMG_3859.jpg
IMG_2892.jpg
IMG_2889.jpg
IMG_2895.jpg
IMG_3858.jpg
IMG_3872.jpg
IMG_3879.jpg
IMG_3871.jpg
IMG_3895.jpg
IMG_3885.jpg
IMG_3899.jpg
IMG_3886.jpg
IMG_3890.jpg
IMG_3888.jpg
IMG_3892.jpg
IMG_3912.jpg
IMG_3934.jpg
IMG_3926.jpg
IMG_3924.jpg
IMG_3944.jpg
IMG_39

In [17]:
import pickle

with open(BASE_PATH+'bin/class8.pickle', 'rb') as f:
    labels = pickle.load(f)

파일 추론하기

In [18]:
import pandas as pd
import cv2
import numpy as np

# --------------------------- Step 4. Loading model to the device -----------------------------------------------------
print('Loading the model to the plugin')
compiled_model = core.compile_model(model, 'CPU')
img_height = 224

result = []

res = open(BASE_PATH+"result.txt", "w")

print('Starting inference in synchronous mode')
cnt = 0;

for file in file_list:
    # --------------------------- Step 5. Set up input --------------------------------------------------------------------
    # Read input image
    image_path = file
    org_image = cv2.imread(image_path)
    image = cv2.resize(org_image, (img_height,img_height), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    #img = cv2.resize(img, (224,224), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.

    # Add N dimension
    nchw_tensor = np.expand_dims(image, 0)

    # NHWC -> NCHW
    input_tensor = np.transpose(nchw_tensor, (0,1,2,3))#(0,3,1,2))

    # --------------------------- Step 6. Create infer request and do inference synchronously -----------------------------
    results = compiled_model.infer_new_request({0:input_tensor})

    # --------------------------- Step 7. Process output ------------------------------------------------------------------
    predictions = next(iter(results.values()))
    output_node_name = next(iter(results.keys())) #

    # Change a shape of a numpy.ndarray with results to get another one with one dimension
    probs = predictions.reshape(-1)

    # Get an array of 8 class IDs in descending order of probability
    top_8 = np.argsort(probs)[-8:][::-1]
    print(top_8)

    header = 'class_id            probability'
    print(f'Image path: {image_path}')
    print('Top 8 results: ')
    print(header)
    print('-' * len(header))

    for class_id in top_8:
        probability_indent = ' ' * (len('class_id           ') - len(labels[class_id]) + 1)
        print(f'{labels[class_id]}{probability_indent}{probs[class_id]*100:.7f}')

    res.write(file+':top1 = ('+labels[top_8[0]]+')'+str(probs[top_8[0]]*100)+', top2 = ('+labels[top_8[1]]+')'+str(probs[top_8[1]]*100)+'\n')

res.close()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Intel               99.9472201
Hp                  0.0385950
Gmarket             0.0108862
Yonsei              0.0014325
Eagle               0.0009672
Wonju               0.0007267
Underwood           0.0001213
Bonobono            0.0000463
[4 2 6 7 3 1 0 5]
Image path: ./drive/MyDrive/competition/drone_real/data/val/Intel/IMG_2791.jpg
Top 8 results: 
class_id            probability
-------------------------------
Intel               99.9639153
Gmarket             0.0252734
Wonju               0.0043295
Yonsei              0.0035836
Hp                  0.0026514
Eagle               0.0002347
Bonobono            0.0000162
Underwood           0.0000043
[4 6 3 7 2 5 1 0]
Image path: ./drive/MyDrive/competition/drone_real/data/val/Intel/IMG_2780.jpg
Top 8 results: 
class_id            probability
-------------------------------
Intel               99.9918222
Wonju               0.0041085
Hp                  0.0029062
Yonsei              0.0007584
Gmarket